In [1]:
from utils import * 
import numpy as np

URL = "http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz"
f_name = "lastfm-dataset-1K.tar.gz"
dir_name = "lastfm-dataset-1K"
dataset_f_name = "userid-timestamp-artid-artname-traid-traname.tsv"

In [2]:
download_dataset(URL, f_name)

Dataset Exists...


In [3]:
user_data = load_dataset(dir_name, dataset_f_name, 1000000)

Read File...


In [4]:
user_data[:10]

,userid,timestamp,artist-id,artist-name,track-id,track-name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Improvisation (Live_2009_4_15)
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Glacier (Live_2009_4_15)
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Parolibre (Live_2009_4_15)
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Bibo No Aozora (Live_2009_4_15)


## Cleaning infrequent data

First remove infrequent tracks (<10 plays)

In [5]:
# Removing infrequent tracks.
if user_data['track-id'].isnull().sum() > 0:
    user_data = user_data.dropna(axis = 0, subset = ['track-id'])

In [6]:
user_data.shape

(865616, 6)

In [7]:
total_plays = user_data.groupby(["track-id"]).size().reset_index()
total_plays.rename(columns = {0: 'total'}, inplace = True)
total_plays.describe()

,total
count,126986.000000
mean,6.816625
std,25.337567
min,1.000000
25%,1.000000
50%,2.000000
75%,6.000000
max,2069.000000


In [8]:
frequent_plays = total_plays[total_plays['total'] >= 10]['track-id']
frequent_plays.describe()

count                                    20666
unique                                   20666
top       8837c142-45e4-495f-abd9-4f3c016dfa52
freq                                         1
Name: track-id, dtype: object

In [9]:
# Drop infrequent tracks.
data = user_data[user_data['track-id'].isin(frequent_plays)]
data.describe()

,userid,timestamp,artist-id,artist-name,track-id,track-name
count,595039,595039,595039,595039,595039,595039
unique,41,592738,3331,3309,20666,19328
top,user_000033,2008-11-24T11:38:21Z,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,82558949-cd98-4c58-af35-3f1a9430d52e,Heartless
freq,58772,24,26616,26616,2069,2120


Now, remove infrequent users.

In [10]:
# Removing infrequent tracks.
if data['userid'].isnull().sum() > 0:
    data = data.dropna(axis = 0, subset = ['userid'])

In [11]:
data.shape

(595039, 6)

In [12]:
user_plays = data.groupby(["userid"]).size().reset_index()
user_plays.rename(columns = {0: 'total'}, inplace = True)
user_plays.describe()

,total
count,41.000000
mean,14513.146341
std,15059.907282
min,176.000000
25%,3090.000000
50%,10437.000000
75%,18144.000000
max,58772.000000


In [13]:
frequent_users = user_plays[user_plays['total'] >= 10]['userid']
frequent_users.describe()

count              41
unique             41
top       user_000035
freq                1
Name: userid, dtype: object

In [14]:
# Drop infrequent tracks.
data = data[data['userid'].isin(frequent_users)]
data.describe()

,userid,timestamp,artist-id,artist-name,track-id,track-name
count,595039,595039,595039,595039,595039,595039
unique,41,592738,3331,3309,20666,19328
top,user_000033,2008-11-24T11:38:21Z,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,82558949-cd98-4c58-af35-3f1a9430d52e,Heartless
freq,58772,24,26616,26616,2069,2120


## Listening Sessions

We create listening sessions for the ground-truth of top-n recommendations and for song2vec

In [15]:
user_play_counts=data.groupby(["userid"]).size().to_frame('size').reset_index()
user_play_counts

,userid,size
0,user_000001,6723
1,user_000002,42930
2,user_000003,10437
3,user_000004,8200
4,user_000005,13211
5,user_000006,10246
6,user_000007,182
7,user_000008,34997
8,user_000009,1857
9,user_000010,3090


In [16]:
groups=data.groupby('userid')
groups.groups

{'user_000001': Int64Index([   10,    12,    14,    15,    74,   174,   255,   256,   257,
               258,
             ...
             16653, 16654, 16655, 16659, 16667, 16669, 16676, 16682, 16683,
             16684],
            dtype='int64', length=6723),
 'user_000002': Int64Index([16685, 16686, 16696, 16697, 16698, 16704, 16705, 16708, 16710,
             16711,
             ...
             74112, 74113, 74114, 74115, 74117, 74118, 74119, 74120, 74121,
             74122],
            dtype='int64', length=42930),
 'user_000003': Int64Index([74127, 74128, 74144, 74147, 74161, 74163, 74164, 74165, 74166,
             74167,
             ...
             93602, 93603, 93605, 93606, 93611, 93612, 93613, 93614, 93615,
             93616],
            dtype='int64', length=10437),
 'user_000004': Int64Index([ 93622,  93627,  93635,  93637,  93638,  93639,  93648,  93649,
              93650,  93651,
             ...
             112007, 112008, 112010, 112016, 112019, 112020, 1

In [17]:
import datetime, dateutil.parser
sessions = []
sessions_user = []
for user, indices in groups.groups.iteritems():
    print("------------------------------------------ User ",user)
    user_record = data.loc[indices, :].sort_values(['timestamp'])
    print user_record.shape
#     print user_record.index
    session = [user_record.iloc[0].name]
    for i in range(0, user_record.shape[0] - 1):
        d1 = int(dateutil.parser.parse(user_record.iloc[i]['timestamp']).strftime('%s'))
        d2 = int(dateutil.parser.parse(user_record.iloc[i+1]['timestamp']).strftime('%s'))
        if(d2-d1 < 800 ):
            session.append(user_record.iloc[i+1].name)
        else:
            if(len(session)>10):
                sessions.append(session)
                sessions_user.append(user)
            session=[]
            session.append(user_record.iloc[i+1].name)
    

('------------------------------------------ User ', 'user_000038')
(7167, 6)
('------------------------------------------ User ', 'user_000039')
(11142, 6)
('------------------------------------------ User ', 'user_000036')
(1471, 6)
('------------------------------------------ User ', 'user_000037')
(2875, 6)
('------------------------------------------ User ', 'user_000034')
(11816, 6)
('------------------------------------------ User ', 'user_000035')
(1213, 6)
('------------------------------------------ User ', 'user_000032')
(12093, 6)
('------------------------------------------ User ', 'user_000033')
(58772, 6)
('------------------------------------------ User ', 'user_000030')
(10939, 6)
('------------------------------------------ User ', 'user_000031')
(12135, 6)
('------------------------------------------ User ', 'user_000014')
(176, 6)
('------------------------------------------ User ', 'user_000015')
(10161, 6)
('------------------------------------------ User ', 'user

## Storing id-name mappings

Forming an artist, track id-name mapping.

In [18]:
artists = data[["artist-id", "artist-name"]].drop_duplicates()


In [19]:
track = data[["track-id", "track-name"]].drop_duplicates()

Forming a mapping from artist to track

In [20]:
artist_track = data[["track-id", "artist-id"]].drop_duplicates()

## Playcount matrix

In [21]:
playcount = data.groupby(["userid", "track-id"]).size().reset_index()

In [22]:
playcount.rename(columns = {0: 'playcount'}, inplace = True)

In [23]:
playcount[:10]

,userid,track-id,playcount
0,user_000001,00b07689-ec4c-4773-94ce-06f3d198431e,4
1,user_000001,0198d56b-a92f-4596-a206-9e136f56e39f,21
2,user_000001,019cc5d9-1aaf-4687-b0af-6bc83bbccf0b,16
3,user_000001,01d866fc-7145-46af-ad7a-4133a477d54b,6
4,user_000001,02d30185-9d38-4f1c-9dac-63aa14f8cc69,1
5,user_000001,03156214-005e-456f-a5e3-0df32024fdad,16
6,user_000001,04049f3b-2fda-460f-a3ea-05f0418792c1,13
7,user_000001,04be389b-e44f-46c3-9ff3-2a0f0c3beda7,12
8,user_000001,05182b53-d942-4e39-9d87-3e62658898ff,14
9,user_000001,05ef9a7c-edab-4937-9c46-6bf572017d14,17


In [24]:
from scipy.sparse import csr_matrix

playcounts = playcount.pivot(index = 'userid', columns = 'track-id', values = 'playcount').fillna(0)


In [25]:
plays_matrix = csr_matrix(playcounts.values)
playcounts[:10]

track-id,0002c0b8-b737-49d8-a007-68d3bc2da175,0003dd36-b4d2-4216-a37e-b110f6882ecb,0005ebe2-6971-4de6-a8eb-5dff41f3b811,00063d3a-14a8-4748-8025-4b7cb828b37f,00087784-820c-4f71-b539-1f948b37befe,0019e275-6d9f-4ff8-bcd8-1bfce804163a,002017be-b745-42e4-aaf2-74a1c3879703,00226a83-ec86-4950-83de-650c3715e344,00271b0b-0459-4984-a3a5-a1f88e1136ca,002a1ae6-34bf-4687-b2ff-f78aaf7c74f5,...,ffddf0b4-d265-40e1-8950-f97207e86838,ffe2102d-4a5e-48b7-977a-0c28e2af119e,ffe3798a-44fd-4795-969c-2e14fc7083c8,ffea36e9-ae42-402d-ba47-2c1ef4cfca05,ffeaac69-dfd9-4159-950f-3981d5775b91,fff02100-41bf-440f-bcb8-f8998b6b1cc7,fff1607f-8763-460f-bf46-44da9c95a93a,fff7464a-d5db-4dd7-9c40-9e8deddf6063,fffada82-2845-44a0-a783-899f383e295e,fffe453c-b68b-4e43-9cef-b6767a587415
userid,,,,,,,,,,,,,,,,,,,,,
user_000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000002,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
user_000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000005,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000006,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
playcounts.index

Index([u'user_000001', u'user_000002', u'user_000003', u'user_000004',
       u'user_000005', u'user_000006', u'user_000007', u'user_000008',
       u'user_000009', u'user_000010', u'user_000011', u'user_000012',
       u'user_000013', u'user_000014', u'user_000015', u'user_000016',
       u'user_000017', u'user_000018', u'user_000019', u'user_000020',
       u'user_000021', u'user_000022', u'user_000023', u'user_000024',
       u'user_000025', u'user_000026', u'user_000027', u'user_000028',
       u'user_000029', u'user_000030', u'user_000031', u'user_000032',
       u'user_000033', u'user_000034', u'user_000035', u'user_000036',
       u'user_000037', u'user_000038', u'user_000039', u'user_000040',
       u'user_000041'],
      dtype='object', name=u'userid')

## Conversion of Playcounts to Ratings

Based on work in A hybrid online-product recommendation system: Combining implicit
rating-based collaborative filtering and sequential pattern analysis by Choi et al.

Absolute Preference: AP(u, i) = ln((n(u played i)/n(u played anything)) + 1)

Relative Preference: RP(u, i) = AP(u,i) / Max_{c \in U} AP(c, i)

Implicit Rating: R(u, i) = Round up(5 * RP (u,i))

In [27]:
# List of number of songs ever played by each user. Reduces computation
user_plays = plays_matrix.sum(axis=1) # Sum all columns
print(user_plays[:20])
print(user_plays.shape)

#number_of_users

[[  6723.]
 [ 42930.]
 [ 10437.]
 [  8200.]
 [ 13211.]
 [ 10246.]
 [   182.]
 [ 34997.]
 [  1857.]
 [  3090.]
 [  3700.]
 [ 47121.]
 [  4220.]
 [   176.]
 [ 10161.]
 [ 11517.]
 [  3810.]
 [ 10566.]
 [ 22334.]
 [  2793.]]
(41, 1)


In [28]:
# Compute Absolute Rating
absolute_preferences = np.log(plays_matrix / user_plays + 1)

In [29]:
absolute_preferences.shape
n_users=absolute_preferences.shape[0]
n_items=absolute_preferences.shape[1]

In [30]:
# 2. List of maximum ratings of each item
max_ratings = absolute_preferences.max(axis=0) # Along columns
print(max_ratings[:10])
print(max_ratings.shape)

[[ 0.00077131  0.00054501  0.00105916 ...,  0.00215983  0.00067887
   0.00037263]]
(1, 20666)


In [31]:
print(max_ratings[max_ratings ==0].size)

0


In [32]:
print(user_plays[user_plays == 0].size)

0


In [33]:
print(max_ratings.shape)

(1, 20666)


In [34]:
# Relative Preference: RP(u, i) = AP(u,i) / Max_{c \in U} AP(c, i)
relative_preferences = absolute_preferences / (max_ratings)
print(absolute_preferences.shape)
print(relative_preferences.shape)

(41, 20666)
(41, 20666)


In [35]:
# Implicit Rating: R(u, i) = Round up(5 * RP (u,i))
implicit_ratings = np.ceil(5 * relative_preferences)

In [36]:
implicit_ratings.max(axis=1)
print(implicit_ratings[:10, :10])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [37]:
implicit_ratings.shape

(41, 20666)

## Test-Train Split

First, for rating matrix

In [38]:
TRAIN_SIZE = 0.80
# Create boolean mask
# np.random creates a vector of random values between 0 and 1
# Those values are filtered to create a binary mask
msk = np.random.rand(implicit_ratings.shape[0],implicit_ratings.shape[1]) < TRAIN_SIZE
r = np.zeros(implicit_ratings.shape)
print (msk.shape)
 
train_ratings = implicit_ratings.copy()
test_ratings = implicit_ratings.copy()
    
train_ratings[msk] = r[msk]
test_ratings[~msk] = r[~msk] # inverse of boolean mask

(41, 20666)


In [39]:
test_ratings[msk].shape

(1, 677300)

In [40]:
print(implicit_ratings[:6, :6])
print(train_ratings[:6, :6])
print(test_ratings[:6, :6])
print(implicit_ratings.shape)
print(train_ratings.shape)
print(test_ratings.shape)

[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  0.]]
(41, 20666)
(41, 20666)
(41, 20666)


In [103]:
np.count_nonzero(test_ratings, axis=1)
# This means we can use test matrix as ground truth

matrix([[ 437],
        [2788],
        [1090],
        [1290],
        [ 458],
        [ 815],
        [  28],
        [ 162],
        [ 399],
        [ 193],
        [ 624],
        [2818],
        [ 675],
        [ 104],
        [ 514],
        [1822],
        [1258],
        [ 630],
        [1121],
        [ 727],
        [2756],
        [1472],
        [ 921],
        [2450],
        [1974],
        [1349],
        [ 204],
        [ 303],
        [1991],
        [1077],
        [3307],
        [ 431],
        [1234],
        [1615],
        [ 291],
        [ 492],
        [ 462],
        [ 542],
        [ 690],
        [1078],
        [ 689]])

Now for listening sessions aka top-n recommendations train set is the same because we just learn the ratings and predict the best.

In [121]:
# We take the test ratings
y_rec = []
# now we need to sort each row
for u in test_ratings:
    indices = np.nonzero(u)[1]
    nz = indices[np.argsort(u[:,indices], axis=None)]
    y_rec.append(nz[::-1])
print len(y_rec)
print y_rec[:1]

41
[array([[ 3311,  2196,  2443, 10648, 10614, 12822,  6682, 15455,  3309,
        18629, 10247, 12990,  3762, 18218, 18004,  3980, 14427,  6145,
         7233,  4395,  6076, 17542, 14327, 13734,  5013,  8462,  5889,
         5510,  5518, 14034,  6944, 10910, 16164,  7128, 11199, 20541,
        11485, 11872,  7160,  1211,  1462,   568,  1283, 11519, 11191,
        14065, 12175, 16928, 20019, 18441, 15050, 12066,  9566, 12062,
        17735,  4740,  4421, 15049, 16805, 19543, 14747, 19750,  2368,
         8524, 16518, 19868,  1677,  2566,  9272, 18844,  5289, 16996,
        15797, 17282,  4777, 16639,   211, 17448, 14189, 16909, 16708,
         8055, 20169, 14937, 12274, 20192, 12808, 17956, 15404,  1775,
         2906, 10658, 18369, 12758, 19102, 18541, 18655, 17681,  8782,
        20330, 18942, 18409, 14280,  7641,  9459,  1817, 13057, 14606,
        14219, 14422, 14580, 14912, 14247, 14567, 14915, 14324, 14548,
        14516, 14926, 14846,    48, 13858, 12828, 12820, 12730, 12651,
  

In [41]:
scoring_ranks = [data.loc[s, ['track-id']].values.flatten() for s in sessions]

In [42]:
# from sets import Set 
# users = Set(sessions_user)
# y_rec = []
# y_rec_label = []
# for u in users:
#     indices = [i for i,U in enumerate(sessions_user) if U == u]
# #     print np.concatenate([scoring_ranks[i] for i in indices])
#     y_rec.append(np.concatenate([scoring_ranks[i] for i in indices]))
#     y_rec_label.append(u)
#     print "%s done..." % u
# print len(y_rec)
# print y_rec

user_000038 done...
user_000039 done...
user_000036 done...
user_000037 done...
user_000034 done...
user_000035 done...
user_000032 done...
user_000033 done...
user_000030 done...
user_000031 done...
user_000015 done...
user_000016 done...
user_000017 done...
user_000010 done...
user_000011 done...
user_000012 done...
user_000013 done...
user_000018 done...
user_000019 done...
user_000029 done...
user_000028 done...
user_000021 done...
user_000020 done...
user_000023 done...
user_000022 done...
user_000025 done...
user_000024 done...
user_000027 done...
user_000026 done...
user_000006 done...
user_000005 done...
user_000004 done...
user_000003 done...
user_000002 done...
user_000001 done...
user_000041 done...
user_000040 done...
user_000009 done...
user_000008 done...
39
[array(['4d5df1a1-eb83-4963-9d3c-e845f86eb8ca',
       '2432b9a3-a80e-415d-b82d-09591d555fba',
       'a40878dc-7aae-41e4-8c91-63582857bfcc', ...,
       'f2f080b7-082e-4aee-a884-2e7a98347d16',
       'fbd7fbe1-5c94-4

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: the sets module is deprecated
  """Entry point for launching an IPython kernel.


In [43]:
len(sessions)

14346

## Metrics

We use the following metrics

### For Rating Prediction

#### Mean Absolute Error

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/3ef87b78a9af65e308cf4aa9acf6f203efbdeded)


#### Root Mean Square Error

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/24a21c4e87218f03bdc76d7b858b93ed448d7682)


### For Top-n Recommendations

#### Precision at k

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/95eeb143dd5ded21c882eaa4530ec69739a3a218)

Here, documents are each record.


#### Mean Average Precision at k

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/64788ba190d585f3837f9d10fe24f3b9689dba4b)

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/decf93706ec7c8632fdfabe41470962101f9bcd8)

#### Truncated Normalized Discounted Cumulative Gain (NDCG@K)

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/c7cf810f95722b0add5a9efb17b47215ada9e168)

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/7f28a12199cde773a6897d5f3021d4e2d3b2f0fe)

In [44]:
from metrics import * 

In [45]:
#type(implicit_ratings)
ratings = np.array(implicit_ratings)
test_r = np.array(test_ratings)
train_r = np.array(train_ratings)

In [46]:
ratings

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  2.,  0., ...,  0.,  0.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [47]:
test_r

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [48]:
train_r

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  2.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [49]:
print (n_users)
print (n_items)
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

41
20666
Sparsity: 6.39%


In [ ]:
import numpy as np
from bmf import *

In [ ]:
# print(rmse)
# print(num_f)

In [ ]:
num_f1=[]
rmse1=[]
it = [20, 40, 60, 80, 100]

for i in it: 
    rmse=[]
    num_f=[]
    bmf = MF(train_r, test_r, K=8, alpha=0.1, beta=0.01, iterations=i)
    training_process = bmf.train()
    rmse.append(float(bmf.mse()))
    num_f.append(bmf.K)
    
    bmf = MF(train_r, test_r, K=16, alpha=0.1, beta=0.01, iterations=i)
    training_process = bmf.train()
    rmse.append(float(bmf.mse()))
    num_f.append(bmf.K)
    
    bmf = MF(train_r, test_r, K=32, alpha=0.1, beta=0.01, iterations=i)
    training_process = bmf.train()
    rmse.append(float(bmf.mse()))
    num_f.append(bmf.K)
    
    bmf = MF(train_r, test_r, K=64, alpha=0.1, beta=0.01, iterations=i)
    training_process = bmf.train()
    rmse.append(float(bmf.mse()))
    num_f.append(bmf.K)
    
    bmf = MF(train_r, test_r, K=128, alpha=0.1, beta=0.01, iterations=i)
    training_process = bmf.train()
    rmse.append(float(bmf.mse()))
    num_f.append(bmf.K)
    rmse1.append(rmse)
    num_f1.append(num_f)
    



In [ ]:
rmse2=[rmse1]
num_f2=[num_f1]

In [ ]:
rmse2.append(rmse1)
num_f1=[]
rmse1=[]
it = [20, 40, 60, 80, 100]
a=0.05
for i in it: 
    rmse=[]
    num_f=[]
    mf = MF(train_r, test_r, K=8, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=16, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=32, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=64, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=128, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    rmse1.append(rmse)
    num_f1.append(num_f)
    num_f2.append(num_f1)

In [ ]:


num_f1=[]
rmse1=[]
it = [20, 40, 60, 80, 100]
a=0.05
for i in it: 
    rmse=[]
    num_f=[]
    mf = MF(train_r, test_r, K=8, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=16, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=32, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=64, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    mf = MF(train_r, test_r, K=128, alpha=a, beta=a, iterations=i)
    training_process = mf.train()
    rmse.append(float(mf.mse()))
    num_f.append(mf.K)
    rmse1.append(rmse)
    num_f1.append(num_f)

In [ ]:
rmse2.append(rmse1)
num_f2.append(num_f1)

In [ ]:
import matplotlib.pyplot as plt

rmse = [1.70891147951593, 1.6766837265777976, 1.6619516094191462, 1.6560941443090653, 1.6563708797542356]
num_f = [8, 16, 32, 64, 128]
plt.plot(num_f, rmse, '--', marker='o', color='b', label='bmf')
plt.xticks(fontsize=8);
plt.yticks(fontsize=8);
plt.xlabel('d(number of factors)', fontsize=8);
plt.ylabel('RMSE', fontsize=10);
plt.legend(loc='best', fontsize=10);
plt.savefig('bmf.png')

In [ ]:
plt.show()

In [ ]:
from msbmf import *

In [55]:
print test_ratings.shape
sim = np.random.normal(scale=1./8, size=(train_ratings.shape[1], 10))

(41, 20666)


In [ ]:
msmf = MSBMF(train_r, test_r, S=sim, D=8,k=3, alpha=1, l=1,eta=0.05, iterations=100)
# R, Rte, S, D, k, alpha, l, eta, iterations)
training_process = msmf.train()
print float(msmf.mse())

In [50]:
import mrec

In [165]:
# from mswrmf import *
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from mrec.sparse import fast_sparse_matrix
from mrec.mf.recommender import MatrixFactorizationRecommender

class MSWRMF(MatrixFactorizationRecommender):
    """
    Parameters
    ==========
    d : int
        Number of latent factors.
    alpha : float
        Confidence weight, confidence c = 1 + alpha*r where r is the observed "rating".
    lbda : float
        Regularization constant.
    num_iters : int
        Number of iterations of alternating least squares.
    """

    def __init__(self,d,k,a, alpha=1,lbda=0.015,num_iters=15):
        self.d = d
        self.k = k
        self.a = a
        self.alpha = alpha
        self.lbda = lbda
        self.num_iters = num_iters

        # R, Rte, S, D, k, alpha, l, eta, iterations

    def __str__(self):
        return 'WRMFRecommender (d={0},alpha={1},lambda={2},num_iters={3})'.format(self.d,self.alpha,self.lbda,self.num_iters)

    def init_factors(self,num_factors,assign_values=True):
        if assign_values:
            return self.d**-0.5*np.random.random_sample((num_factors,self.d))
        return np.empty((num_factors,self.d))

    def fit(self,train,S,item_features=None):
        """
        Learn factors from training set. User and item factors are
        fitted alternately.
        Parameters
        ==========
        train : scipy.sparse.csr_matrix or mrec.sparse.fast_sparse_matrix
            User-item matrix.
        S: item similaities
        item_features : array_like, shape = [num_items, num_features]
            Features for each item in the dataset, ignored here.
        """
        self.S = S
        nbrs = NearestNeighbors(n_neighbors=self.k + 1).fit(self.S)
        # Does contain itself, hence k + 1
        self.k_distances, self.k_indices = nbrs.kneighbors(self.S)

        if type(train) == csr_matrix:
            train = fast_sparse_matrix(train)

        num_users,num_items = train.shape

        self.U = self.init_factors(num_users,False)  # don't need values, will compute them
        self.V = self.init_factors(num_items) # Items
        for it in xrange(self.num_iters):
            print 'iteration',it
            # fit user factors
            VV = self.V.T.dot(self.V)
            for u in xrange(num_users):
                # get (positive i.e. non-zero scored) items for user
                indices = train.X[u].nonzero()[1]
                if indices.size:
                    self.U[u,:] = self.update(indices,self.V,VV)
                else:
                    self.U[u,:] = np.zeros(self.d)
            # fit item factors
            UU = self.U.T.dot(self.U)
            for i in xrange(num_items):
                indices = train.fast_get_col(i).nonzero()[0]
                if indices.size:
                    self.V[i,:] = self.update_item(indices,self.U,UU)
                else:
                    self.V[i,:] = np.zeros(self.d)

    # # Backprop rule also contains si,j term hiii
    #         similarity_factor = np.zeros_like(self.Q[j, :])
    #         for x, d in enumerate(self.k_distances[j]):
    #             similarity_factor += (d - self.Q[j, :].dot(self.Q[self.k_indices[i][x], :].T)) * (self.Q[self.k_indices[i][x], :])
    #         self.Q[j, :] += self.eta * (e * self.P[i, :] - self.l * self.Q[j,:] - self.alpha * similarity_factor)

    #  NOW WE NEED A DIFFERENT UPDATE RULE FOR ITEM AND USERS

    def update(self,indices,H,HH):
        """
        Update latent factors for a single user.
        """
        Hix = H[indices,:]
        M = HH + self.alpha*Hix.T.dot(Hix) + np.diag(self.lbda*np.ones(self.d))
        return np.dot(np.linalg.inv(M),(1+self.alpha)*Hix.sum(axis=0))

    def update_item(self,indices,H,HH):
        """
        Update latent factors for a single item.
        """
        Hix = H[indices,:]
        for x, d in enumerate(self.k_distances[j]):
            Hix -= self.a * self.V[j, :].dot(self.V[self.k_indices[i][x], :].T) * (self.V[self.k_indices[i][x], :])
        M = HH + self.alpha*Hix.T.dot(Hix) + np.diag(self.lbda*np.ones(self.d))
        for x, d in enumerate(self.k_distances[j]):
            M -= self.a * d * (self.V[self.k_indices[i][x], :])
        return np.dot(np.linalg.inv(M),(1+self.alpha)*Hix.sum(axis=0))



In [166]:
# msmf = MSBMF(train_r, test_r, S=sim, D=8,k=3, alpha=1, l=1,eta=0.05, iterations=100)
mswrmf = MSWRMF(d=8,k=5, a=0.1, num_iters=10)
# R, Rte, S, D, k, alpha, l, eta, iterations)
from scipy.sparse import csr_matrix
training_process = mswrmf.fit(csr_matrix(train_r), sim)
# print float(msmf.mse())

iteration 0


NameError: global name 'j' is not defined

In [122]:
n_rec = mswrmf.batch_recommend_items(csr_matrix(train_r),max_items=10,return_scores=False)

In [123]:
n_rec

[[19501, 8017, 16060, 18907, 11273, 1561, 9687, 3940, 2243, 7994],
 [14807, 11704, 20041, 505, 8294, 15591, 9368, 13908, 13061, 15982],
 [13924, 121, 4777, 8633, 8462, 19830, 13109, 20080, 15501, 7923],
 [19897, 5252, 9687, 7394, 2374, 2935, 18295, 13645, 4336, 1299],
 [505, 15591, 19830, 18052, 10256, 8294, 20205, 2396, 3028, 870],
 [17561, 11704, 13061, 20196, 14486, 1313, 11227, 8579, 6847, 583],
 [7994, 13924, 19501, 18097, 16323, 10740, 3531, 11273, 8531, 17038],
 [2556, 2501, 6839, 505, 17268, 1216, 10788, 19185, 9687, 15248],
 [13924, 10993, 2396, 121, 2865, 10873, 505, 9577, 13639, 2223],
 [2396, 505, 13924, 20076, 121, 20080, 15501, 12777, 2556, 10898],
 [505, 19786, 18641, 7037, 2944, 9455, 3773, 1379, 9687, 10993],
 [3180, 15591, 4006, 14807, 2935, 10806, 11638, 18579, 55, 15048],
 [9687, 19185, 3940, 14392, 1561, 13061, 3501, 3773, 1313, 8147],
 [20076, 7443, 505, 15799, 4682, 5600, 8129, 6258, 13536, 3951],
 [9687, 505, 19786, 13307, 19185, 17561, 3940, 13924, 121, 18844],

In [94]:
len(n_rec)

41

In [124]:
# Now we need to evaluate it y_rec vs n_rec
#  playcounts.index corresponds to n_rec; y_rec_label to y_rec
n_rec_label = list(playcounts.index)
# So we reshuffle n_recs to correspond to y_rec order
# So we iter over y_rec_label and find where index of each instance in list
label_index = []
for l in y_rec_label:
    label_index.append(n_rec_label.index(l))
    
# print label_index
ordered_n_rec = [n_rec[i] for i in label_index]

In [163]:
from metrics import *

In [164]:
# from metrics import *
print ("MAP@k",mean_average_precision(y_rec, ordered_n_rec, 10))
print ("P@k",precision_k(y_rec, ordered_n_rec, 10))
print ("NDCG@k",ndcg_k(y_rec, ordered_n_rec, 10))

('MAP@k', 0.49000814000813991)
('P@k', 0.12307692307692307)
('NDCG@k', 0.96654800564566812)
